**Homework 2, due Friday, January 23**

In [8]:
%pip install ortools

# Problem 1

**Winston, p. 554, Problem 15.** Solve the problem using the following code outline. Use comments to explain the meaning of your variables, constraints, and anything else that might be unclear.

In [9]:
from ortools.linear_solver import pywraplp

# We use the integer program solver SAT instead of GLOP.
solver = pywraplp.Solver.CreateSolver("SAT")

# Define the variables. ADD A COMMENT explaining the meaning of your variables.
x1 = solver.IntVar(0, 1, "x1")
x2 = solver.IntVar(0, 1, "x2")
x3 = solver.IntVar(0, 1, "x3")
x4 = solver.IntVar(0, 1, "x4")
x5 = solver.IntVar(0, 1, "x5")
x6 = solver.IntVar(0, 1, "x6")


# Set the objective.
solver.Minimize(x1 + x2 + x3 + x4 + x5 + x6)


# Add the constraints.
solver.Add(x1 + x4 >= 1)          # Operation 1: can be done by surgeon 1 or 4
solver.Add(x1 + x5 >= 1)          # Operation 2: can be done by surgeon 1 or 5
solver.Add(x2 + x3 >= 1)          # Operation 3: can be done by surgeon 2 or 3
solver.Add(x1 + x6 >= 1)          # Operation 4: can be done by surgeon 1 or 6
solver.Add(x2 + x3 + x6 >= 1)     # Operation 5: can be done by surgeon 2, 3, or 6
solver.Add(x2 + x4 >= 1)

# surgeons 1 and 2 cannot both be on duty
solver.Add(x1 + x2 <= 1)


# Solve and print the solution. DO NOT CHANGE ANYTHING PAST THIS POINT.
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
  print("Solution:")
  print(f"Optimal value = {solver.Objective().Value():0.0f}")
  print(f"{x1} = {x1.solution_value():0.0f}")
  print(f"{x2} = {x2.solution_value():0.0f}")
  print(f"{x3} = {x3.solution_value():0.0f}")
  print(f"{x4} = {x4.solution_value():0.0f}")
  print(f"{x5} = {x5.solution_value():0.0f}")
  print(f"{x6} = {x6.solution_value():0.0f}")
else:
  print("The problem does not have an optimal solution.")

Solution:
Optimal value = 3
x1 = 1
x2 = 0
x3 = 1
x4 = 1
x5 = 0
x6 = 0


# Problem 2

**Winston, p. 503, Problem 14.** Solve the problem using the following code outline. We will introduce using lists to more easily store and handle variables and constants. It may help to look at "meetingplanner.py" on Github and/or https://developers.google.com/optimization/lp/stigler_diet for more examples of how to handle lists in OR-Tools.

In [10]:
solver = pywraplp.Solver.CreateSolver("SAT")

# The variables are x[j], where x[j] = 1 if disk j+1 is chosen and otherwise.
# Because Python indices start at 0 but the problem indices start at 1, we
# let x[j] correspond to disk j+1. The following code creates a list of these
# variables. DO NOT CHANGE THIS LINE. (The "" is fine.)
x = [solver.IntVar(0, 1, "") for j in range(10)]



# Before proceeding, we will create some lists of constants. This will make
# defining our objective and constraints easier. COMPLETE THE BELOW DEFINITIONS
# USING THE INFORMATION FROM THE PROBLEM.

# Storage required by each disk.
storage = [3, 5, 1, 2, 1, 4, 3, 1, 2, 2] # COMPLETE THIS.

# a[i][j] = 1 if file i+1 is on disk j+1, and 0 otherwise.
a = [[1,1,0,1,1,0,0,1,1,0],
     [1,0,1,0,0,0,0,0,0,0],
     [0,1,0,0,1,0,1,0,0,1],
     [0,0,1,0,0,1,0,1,0,0],
     [1,1,0,1,0,1,1,0,1,1]]




# Set the objective. We will use the Sum method provided by OR-Tools to do this.
# DO NOT CHANGE THIS.
solver.Minimize(solver.Sum(storage[j]*x[j] for j in range(10)))



# Add the constraints that each file must be on at least one chosen disk. We
# can do this with one for-loop. COMPLETE THIS, without adding additional lines.
for i in range(5):
  solver.Add(solver.Sum(a[i][j]*x[j] for j in range(10)) >= 1)



# Solve and print the solution to (a). DO NOT CHANGE ANYTHING IN THIS BLOCK.
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
  print("Part (a) solution:")
  print(f"Minimum storage = {solver.Objective().Value():0.0f}K")
  print(f"Disks chosen = {[j+1 for j in range(10) if x[j].solution_value() == 1]}")
else:
  print("The problem does not have an optimal solution.")



# Add the constraint for part (b) using one line. Remember, you can use x[j] to
# call the variable for disk j+1.
solver.Add(x[2] <= x[1])
solver.Add(x[4] <= x[1])



# Solve and print the solution to (b). DO NOT CHANGE ANYTHING IN THIS BLOCK.
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
  print("Part (b) solution:")
  print(f"Minimum storage = {solver.Objective().Value():0.0f}K")
  print(f"Disks chosen = {[j+1 for j in range(10) if x[j].solution_value() == 1]}")
else:
  print("The problem does not have an optimal solution.")


Part (a) solution:
Minimum storage = 4K
Disks chosen = [3, 8, 10]
Part (b) solution:
Minimum storage = 6K
Disks chosen = [2, 3]


# Problem 3

**Winston, p.554, Problem 17.** Solve the problem using the following code outline. Assume you can only purchase an integer amount of each type of steel ingot, while you can purchase any amount of alloy and steel scrap, including fractions of tons.

In [11]:
# We will use the mixed integer program solver SCIP.
solver = pywraplp.Solver.CreateSolver("SCIP")

# Define the variables. Make sure to use NumVar for real variables and IntVar
# for integer variables.
i1 = solver.IntVar(0, 1, "i1")
i2 = solver.IntVar(0, 1, "i2")
i3 = solver.IntVar(0, 1, "i3")
i4 = solver.IntVar(0, 1, "i4")

a1 = solver.NumVar(0, solver.infinity(), "a1")
a2 = solver.NumVar(0, solver.infinity(), "a2")
a3 = solver.NumVar(0, solver.infinity(), "a3")

ss = solver.NumVar(0, solver.infinity(), "ss")

# Set the objective.
solver.Minimize(
    350*5*i1 + 330*3*i2 + 310*4*i3 + 280*6*i4 +
    500*a1 + 450*a2 + 400*a3 +
    100*ss
)


# Add the constraints.
solver.Add(5*i1 + 3*i2 + 4*i3 + 6*i4 + a1 + a2 + a3 + ss == 25)

solver.Add(0.05*5*i1 + 0.04*3*i2 + 0.05*4*i3 + 0.03*6*i4 +
           0.08*a1 + 0.07*a2 + 0.06*a3 + 0.03*ss == 0.05*25)

solver.Add(0.03*5*i1 + 0.03*3*i2 + 0.04*4*i3 + 0.04*6*i4 +
           0.06*a1 + 0.07*a2 + 0*a3 + 0.09*ss == 0.05*25)


# Solve and print the solution. DO NOT CHANGE ANYTHING PAST THIS POINT.
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
  print("Solution:")
  print(f"Optimal value = {solver.Objective().Value():0.2f}")
  print(f"{i1} = {i1.solution_value():0.0f}")
  print(f"{i2} = {i2.solution_value():0.0f}")
  print(f"{i3} = {i3.solution_value():0.0f}")
  print(f"{i4} = {i4.solution_value():0.0f}")
  print(f"{a1} = {a1.solution_value():0.2f}")
  print(f"{a2} = {a2.solution_value():0.2f}")
  print(f"{a3} = {a3.solution_value():0.2f}")
  print(f"{ss} = {ss.solution_value():0.2f}")
else:
  print("The problem does not have an optimal solution.")

Solution:
Optimal value = 7083.33
i1 = 0
i2 = 0
i3 = 0
i4 = 0
a1 = 4.17
a2 = 0.00
a3 = 9.72
ss = 11.11


# Problem 4

**Winston, p. 555, Problem 21.** Solve the problem using the following code outline. **NOTE**: Table 100 has a typo in one entry. Explain what that typo is below, and make sure to use the corrected data in your program.


**Explanation of typo:**
The entry for the travel time from district 1 to itself (row 1, column 1) is 10 minutes. This should be 0 minutes, as it takes 0 time to travel from a district to itself. All other diagonal entries are 0.



In [12]:
solver = pywraplp.Solver.CreateSolver("SAT")

# The variables are defined for you. WRITE COMMENTS to describe what these
# variables mean.

# x[i] = 1 if an ambulance is located in district i+1, 0 otherwise
x = [solver.IntVar(0,1,"") for i in range(8)]

# y[i] = 1 if district i+1 is covered (within 2 minutes of an ambulance), 0 otherwise
y = [solver.IntVar(0,1,"") for i in range(8)]


# List containing the district populations.
population = [40, 30, 35, 20, 15, 50, 45, 60]

# Optional: Define any additional constants you would like to use here.
# Travel time matrix (corrected: district 1 to itself is 0, not 10)
time = [
    [0, 3, 4, 6, 8, 9, 8, 10],
    [3, 0, 5, 4, 8, 6, 12, 9],
    [4, 5, 0, 2, 2, 3, 5, 7],
    [6, 4, 2, 0, 3, 2, 5, 4],
    [8, 8, 2, 3, 0, 2, 2, 4],
    [9, 6, 3, 2, 2, 0, 3, 2],
    [8, 12, 5, 5, 2, 3, 0, 2],
    [10, 9, 7, 4, 4, 2, 2, 0]
]



# Set the objective using the Sum method.
solver.Maximize(solver.Sum(population[i]*y[i] for i in range(8)))


# Add the constraints.
solver.Add(solver.Sum(x[i] for i in range(8)) == 2)
for i in range(8):
    solver.Add(y[i] <= solver.Sum(x[j] for j in range(8) if time[i][j] <= 2))


# Solve and print the solution. DO NOT CHANGE ANYTHING PAST THIS POINT.
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
  print("Solution:")
  print(f"Population within 2 minutes of ambulance = {solver.Objective().Value():0.0f}")
  print(f"Ambulance locations = {[i+1 for i in range(8) if x[i].solution_value() == 1]}")
else:
  print("The problem does not have an optimal solution.")

Solution:
Population within 2 minutes of ambulance = 225
Ambulance locations = [3, 8]


**Submission:** As usual, make sure you click "Run all" before submission, and make sure all the output you want to submit is shown.